In [ ]:
import sys
import os
import pandas as pd

# Caminho absoluto para a pasta src
src_path = os.path.abspath(os.path.join(os.getcwd(), "../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
df_zonas_sp = pd.read_csv('data/subprefeituras_sp_lat_lon_zona.csv')
df_zonas_sp.head()

,nome,latitude,longitude,zona
0,Sé,-23.5505,-46.6333,Centro
1,Pinheiros,-23.5675,-46.6931,Oeste
2,Lapa,-23.5273,-46.7036,Oeste
3,Vila Mariana,-23.5898,-46.6346,Centro-Sul
4,Butantã,-23.5728,-46.7220,Oeste


In [10]:
df_zonas_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nome       33 non-null     object 
 1   latitude   33 non-null     float64
 2   longitude  33 non-null     float64
 3   zona       33 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.2+ KB


In [11]:
from services.api_service import extrair_dados
import time 
start_date = "2000-01-01"
end_date = "2024-12-01"
dfs = []
for _, row in df_zonas_sp.iterrows():
    print(f"Extraindo dados para: {row['nome']}")

    try:
        df = extrair_dados(
            latitude=row["latitude"],
            longitude=row["longitude"],
            start_date=start_date,
            end_date=end_date
        )
        df["subprefeitura"] = row["nome"]
        df["zona"] = row["zona"]
        dfs.append(df)
        time.sleep(60)
    except Exception as e:
        print(f"Erro ao extrair dados para {row['nome']}: {e}")
df_final = pd.concat(dfs, ignore_index=True)

Extraindo dados para: Sé
Extraindo dados para: Pinheiros
Extraindo dados para: Lapa
Extraindo dados para: Vila Mariana
Extraindo dados para: Butantã
Extraindo dados para: Casa Verde/Cachoeirinha
Extraindo dados para: Santana/Tucuruvi
Extraindo dados para: Jaçanã/Tremembé
Extraindo dados para: Freguesia do Ó/Brasilândia
Extraindo dados para: Perus
Extraindo dados para: Pirituba/Jaraguá
Extraindo dados para: Campo Limpo
Extraindo dados para: Capela do Socorro
Extraindo dados para: Cidade Ademar
Extraindo dados para: Cidade Tiradentes
Extraindo dados para: Ermelino Matarazzo
Extraindo dados para: Guaianases
Extraindo dados para: Itaim Paulista
Extraindo dados para: Itaquera
Extraindo dados para: Jabaquara
Extraindo dados para: Mooca
Extraindo dados para: Parelheiros
Extraindo dados para: Penha
Extraindo dados para: Santana
Extraindo dados para: Santo Amaro
Extraindo dados para: São Mateus
Extraindo dados para: São Miguel
Extraindo dados para: Sapopemba
Extraindo dados para: Tatuapé
Extrai

In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208784 entries, 0 to 7208783
Data columns (total 7 columns):
 #   Column            Dtype              
---  ------            -----              
 0   datahora          datetime64[ns, UTC]
 1   temperatura       float64            
 2   sensacao_termica  float64            
 3   latitude          float64            
 4   longitude         float64            
 5   subprefeitura     object             
 6   zona              object             
dtypes: datetime64[ns, UTC](1), float64(4), object(2)
memory usage: 385.0+ MB


In [ ]:
df_final.to_csv('data/all_data.csv', index=False)

In [15]:
from services.bq_service import BigQueryService
bq = BigQueryService()
bq.carregar_dataframe(df_final, "clima_historico_subprefeituras_sp",write_disposition="WRITE_TRUNCATE")

Dados carregados na tabela fluent-century-298900.mba_eng_dados_mack.clima_historico_subprefeituras_sp
